In [ ]:
pip install langchain

In [ ]:
!pip install ctransformers==0.2.5
!pip install sentence-transformers==2.2.2
!pip install chromadb

In [ ]:
pip install langchain[all]

In [ ]:
pip install -U langchain-community

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
! pip install pypdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf("E:\Projects\GenAI\AI-Powered Health Companion")

In [ ]:
extracted_data

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

In [ ]:
text_chunks

In [ ]:
pip install --upgrade sentence-transformers huggingface_hub

In [ ]:
import chromadb
chroma_client = chromadb.Client()

In [ ]:
collection = chroma_client.create_collection(name="my_collection")

In [ ]:
documents_to_add = [{"metadata": doc.metadata, "page_content": doc.page_content} for doc in text_chunks]

collection.add(
    documents=[doc["page_content"] for doc in documents_to_add],
    metadatas=[doc["metadata"] for doc in documents_to_add],
    ids=[f"id{i}" for i in range(len(documents_to_add))]  # Unique IDs for each document
)

In [ ]:
# Usage of ChromaDB
results = collection.query(
    query_texts=["What is cervix?"], 
    n_results=1 # how many results to return
)
print(results)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
prompt=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": prompt}

In [ ]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
result = llm("What is AI?")
print(result)


In [ ]:
 embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores import Chroma

retriever = Chroma(collection_name="my_collection",embedding_function=embeddings).as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True,
)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa_chain({"query": user_input})
    print("Response : ", result["result"])